In [1]:
#magic command to display matplotlib plots 
%matplotlib inline

In [2]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Import the BLS Data API key
from api_keys import api_key
import matplotlib.pyplot as plt
from pathlib import Path

ModuleNotFoundError: No module named 'api_keys'

In [ ]:
# Store file paths in a variable
occupation_data_file = Path("Resources/national_M2022_dl.xlsx")
employment_data_file = Path("Resources/occupation.xlsx")
automation_file = Path("Resources/automation_data_by_state.csv")
occupation_gender_file = Path("Resources/cpsaat11.xlsx")

In [ ]:
# Read the files into dataframes
occupation_group_data = pd.read_excel(occupation_data_file)
employment_data_df = pd.read_excel(employment_data_file, sheet_name ="Table 1.7",header=1)
factors_utilization= pd.read_excel(employment_data_file, sheet_name ="Table 1.12",header=1)
automation_data = pd.read_csv(automation_file, encoding="ISO-8859-1")
occupation_gender_df = pd.read_excel(occupation_gender_file,header=None)

In [ ]:
# Sample data of occupation_group_data dataframe
occupation_group_data.sample(5)

In [ ]:
# Columns
occupation_group_data.columns

In [ ]:
# Select the requred columns-'OCC_CODE', 'OCC_TITLE', 'O_GROUP'
occupation_group_data = occupation_group_data[['OCC_CODE', 'OCC_TITLE', 'O_GROUP']]

In [ ]:
# Rename the columns
occupation_group_data.rename(columns={'OCC_CODE':"2022 National Employment Matrix code",'OCC_TITLE' : "2022 National Employment Matrix title",'O_GROUP':"Occupation Group" },inplace=True)


In [ ]:
# Count of each column items
occupation_group_data.count()

In [ ]:
# Check for duplicate records
occupation_group_data.duplicated().sample(5)

In [ ]:
# Reset the index of employment_data_df dataframe
employment_data_df.reset_index()
employment_data_df.head()

In [ ]:
# Display the columns
employment_data_df.columns

In [ ]:
# Count of each column items
employment_data_df.count()

In [ ]:
# Remove the footer in the last 4 lines
employment_data_df = employment_data_df.drop(employment_data_df.index[-4:], axis=0)
employment_data_df.tail()

In [ ]:
# Count of each column items
employment_data_df.count()

In [ ]:
# Select the required columns - 2022 National Employment Matrix occupation code ,Factors affecting occupational utilization
factors_utilization =  factors_utilization[['2022 National Employment Matrix occupation code','Factors affecting occupational utilization']]

In [ ]:
factors_utilization.head()


In [ ]:
# Rename Columns
factors_utilization.rename(columns={'2022 National Employment Matrix occupation code':'2022 National Employment Matrix code'},inplace = True)

In [ ]:
# Remove duplicate records
factors_utilization = factors_utilization.drop_duplicates(subset='2022 National Employment Matrix code', keep='first')

In [ ]:
# Merge the dataframes
employment_details = pd.merge(employment_data_df,factors_utilization,on ='2022 National Employment Matrix code',how="left")

In [ ]:
# Remove the duplicate records
employment_details = employment_details.drop_duplicates(subset="2022 National Employment Matrix code",keep='last')

In [ ]:
# Count
employment_details.count()

In [ ]:
# merge the occupation_group_data and employment_data_df dataframes
employment_wage_education = pd.merge(occupation_group_data,employment_details,on= ["2022 National Employment Matrix code","2022 National Employment Matrix code"])
employment_wage_education.head()

In [ ]:
employment_wage_education.count()

In [ ]:
# Drop the 2022 National Employment Matrix title_y column
employment_wage_education = employment_wage_education.drop("2022 National Employment Matrix title_y",axis=1)

# Rename the column 2022 National Employment Matrix title_x to 2022 National Employment Matrix title
employment_wage_education.rename(columns={"2022 National Employment Matrix title_x":"2022 National Employment Matrix title"},inplace = True)

In [ ]:
# Count
employment_wage_education.count()

In [ ]:
# Display some sample data of automation_data dataframe
automation_data.sample(5)

In [ ]:
# Rename the columns "SOC" to "2022 National Employment Matrix code" and "Probability" to "Automation Probability"
automation_data.rename(columns={"SOC":"2022 National Employment Matrix code","Probability":"Automation Probability"},inplace =True) 

In [ ]:
automation_data = automation_data[["2022 National Employment Matrix code","Automation Probability"]]

In [ ]:
# Merge the employment_wage_education and automation_data dataframes
occupation_automation_df = pd.merge(employment_wage_education,automation_data,how="left",on=["2022 National Employment Matrix code","2022 National Employment Matrix code"])
occupation_automation_df.head()

In [ ]:
# Display the count of column items
occupation_automation_df.count()

In [ ]:
# sample data of occupation_gender_df dataframe
occupation_gender_df.reset_index()

occupation_gender_df.head()

In [ ]:
# Exculde the header rows
occupation_gender_df = occupation_gender_df.iloc[9:]

In [ ]:
# Name columns
occupation_gender_df.columns = ["2022 National Employment Matrix code","2022 National Employment Matrix title","Total employed", "Women","White","Black or African American","Asian","Hispanic or Latino"]

In [ ]:
# Merge occupation_automation_df,occupation_gender_df dataframes
occupation_data_complete = pd.merge(occupation_automation_df,occupation_gender_df,how = "left",on="2022 National Employment Matrix code")


In [ ]:
occupation_data_complete.columns

In [ ]:
# Drop the 2022 National Employment Matrix title_y column
occupation_data_complete = occupation_data_complete.drop("2022 National Employment Matrix title_y",axis=1)

# Rename the column 2022 National Employment Matrix title_x to 2022 National Employment Matrix title
occupation_data_complete.rename(columns={"2022 National Employment Matrix title_x":"2022 National Employment Matrix title"},inplace = True)

In [ ]:
occupation_data_complete["Median annual wage, 2022(1)"]=pd.to_numeric(occupation_data_complete["Median annual wage, 2022(1)"])

In [ ]:
occupation_data_complete.head()

In [ ]:
# Export file as a CSV, without the Pandas index, but with the header
occupation_data_complete.to_csv("Resources/Occupation_complete.csv", index=False, header=True)

In [ ]:
# Create a dataframe for all the major sectors
major_sectors_df= occupation_data_complete.loc[occupation_data_complete['Occupation Group'] == "major" ]
major_sectors_df.count()

In [ ]:
Employment_distribution_df =major_sectors_df[["2022 National Employment Matrix title",'Employment, 2022','Employment, 2032']]

Employment_distribution_df["2022 National Employment Matrix title"] = Employment_distribution_df["2022 National Employment Matrix title"].str.replace("Occupations","")
Employment_distribution_df = Employment_distribution_df.set_index("2022 National Employment Matrix title")
Employment_distribution_df

In [ ]:
# Plot the employments across the job groups
ax =Employment_distribution_df.plot(kind="bar",xlabel='Major Employment Sectors',ylabel = "Employment (Numbers in thousands)",figsize=(8,5))
# Save the figure
plt.savefig("Outputs/Employment_Distribution.png")


In [ ]:
# Major job groups
major_sectors_selected = major_sectors_df[["2022 National Employment Matrix title","Employment change, percent, 2022-32"]]
major_sectors_selected=major_sectors_selected.sort_values("Employment change, percent, 2022-32",ascending = False)
major_sectors_selected["2022 National Employment Matrix title"] = major_sectors_selected["2022 National Employment Matrix title"].str.replace("Occupations","") 
top_occ_groups=major_sectors_selected.head(6)
top_occ_groups

top_occ_list = top_occ_groups["2022 National Employment Matrix title"].to_list()
top_occ_values = top_occ_groups["Employment change, percent, 2022-32"].to_list()


In [ ]:
major_sectors_selected

In [ ]:
bottom_occ_groups = major_sectors_selected.loc[major_sectors_selected["Employment change, percent, 2022-32"]<0]
# .sort_values("Employment change, percent, 2022-32")

bottom_occ_list = bottom_occ_groups["2022 National Employment Matrix title"].to_list()
bottom_occ_values = bottom_occ_groups["Employment change, percent, 2022-32"].to_list()


In [ ]:
# Percentage employment Change for all the occupations
percentage_change_all = employment_data_df[employment_data_df["2022 National Employment Matrix code"]=="00-0000"]

In [ ]:
merged_occ_list = top_occ_list + bottom_occ_list
merged_values_list = top_occ_values+bottom_occ_values


In [ ]:

data = {'Occupational Groups':merged_occ_list,
        'Percentage Change': merged_values_list}

# Create a sample DataFrame

df = pd.DataFrame(data)

# Sorting in ascending order based on the 'Age' column
df_sorted_ascending = df.sort_values(by='Percentage Change')

# Create a horizontal bar plot with negative values in red
fig, ax = plt.subplots()
colors = ['red' if val < 0 else 'green' for val in df_sorted_ascending['Percentage Change']]
bars = ax.barh(df_sorted_ascending['Occupational Groups'], df_sorted_ascending['Percentage Change'], color=colors)

# Add labels and title
ax.set_xlabel('Employment Percentage Change')
ax.set_ylabel('Occupational Groups')
ax.set_title('Employment percentage change projected over the next decade')

for i, v in enumerate(df_sorted_ascending['Percentage Change'].to_list()):
    label = f"{v}%"
    plt.text(v, i, label, va='center')
    
v_label = f"<--Employment change of all occupations {percentage_change_all.iloc[0,6]}%"
ax.axvline(x=percentage_change_all.iloc[0,6], color='yellow', linestyle='--', label=percentage_change_all.iloc[0,6])
ax.text(19, 9.5,v_label , rotation=0, ha='right',color="blue")

# Save the figure
plt.savefig("Outputs/Most_affected_OCCGroups.png")

# Show the plot
plt.show()

In [ ]:
# Occupations that are highly impacted by automation
occupation_detailed = occupation_data_complete[occupation_data_complete['Occupation Group']=="detailed"]
occupation_detailed.columns
occupation_automation = occupation_detailed[['2022 National Employment Matrix title',"Automation Probability"] ]

occupation_automation.dropna()
occupation_automation_sorted = occupation_automation.sort_values("Automation Probability",ascending=False)
occupation_automation_sorted.head()


In [ ]:
# Fastest growing occupations- employment change percenatge > 18%
fastest_growing_occ  =occupation_detailed.loc[occupation_detailed['Employment change, percent, 2022-32'] > 18.0]
fastest_growing_occ_df = fastest_growing_occ[["2022 National Employment Matrix title",'Employment change, percent, 2022-32','Employment change, numeric, 2022-32']]
fastest_growing_occ_top10 = fastest_growing_occ_df.sort_values('Employment change, percent, 2022-32', ascending=False).head(10)

In [ ]:

# Create a horizontal bar plot with negative values in red
fastest_growing_occ_top10 = fastest_growing_occ_top10.sort_values(by='Employment change, percent, 2022-32',ascending = True)

plt.barh(fastest_growing_occ_top10["2022 National Employment Matrix title"], fastest_growing_occ_top10['Employment change, percent, 2022-32'], color="blue")

for i, v in enumerate(fastest_growing_occ_top10['Employment change, percent, 2022-32'].to_list()):
    label = f"{v}%"
    plt.text(v, i, label, va='center')

# Add labels and title

plt.xlabel('Projected employment change 2022-32 (in percentage)')
plt.ylabel('Occupations')
# plt.title('Top 10 Fastest Growing Occupations')

# Save the figure
plt.savefig("Outputs/Fastest_Growing_Occ_Percentage.png")

# Show the plot
plt.show()


In [ ]:
fastest_growing_occ_top10

In [ ]:

fastest_growing_occ_top10_new = fastest_growing_occ_top10.sort_values(by='Employment change, percent, 2022-32',ascending = True)

plt.barh(fastest_growing_occ_top10_new["2022 National Employment Matrix title"], fastest_growing_occ_top10_new['Employment change, numeric, 2022-32'], color="green")

for i, v in enumerate(fastest_growing_occ_top10_new['Employment change, numeric, 2022-32'].to_list()):
    label = f"{v}"
    plt.text(v, i, label, va='center')

# Add labels and title

plt.xlabel('Projected employment change 2022-32 (in thousands)')
plt.ylabel('Occupations')
# plt.title('Top 10 Fastest Growing Occupations')

# Save the figure
plt.savefig("Outputs/Fastest_Growing_Occ_Numeric.png")


# Show the plot
plt.show()

In [ ]:
# Fastest growing occupations
fastest_growing_occ_num = fastest_growing_occ[["2022 National Employment Matrix title",'Employment change, numeric, 2022-32','Employment change, percent, 2022-32']]

In [ ]:
fastest_growing_occ_num

In [ ]:
# Fastest declining occupations- employment change percenatge < 0%
fastest_declining_occ  =occupation_detailed.loc[occupation_detailed['Employment change, percent, 2022-32'] < 0.0]
fastest_declining_occ = fastest_declining_occ[["2022 National Employment Matrix title",'Employment change, percent, 2022-32']]
fastest_declining_occ.sort_values('Employment change, percent, 2022-32', ascending=True).head(15)

In [ ]:
# base url of BLS Data API
base_url = "https://api.bls.gov/publicAPI/v2/"
# Series Id for job openings in Health care and social assistance
series_id = "JTU620000000000000JOL"

#url
url = f"{base_url}timeseries/data/{series_id}"
params = {
    "startyear": 2010,
    "endyear": 2022,
    "registrationkey": api_key
}

#run a request using our params dictionary
openings_response = requests.get(url, params=params)
# convert response to json
openings_response_json = openings_response.json()

# Print the json
print(json.dumps(openings_response_json ,indent=4, sort_keys=True))


In [ ]:
#store the year and openings data in lists
year_list=[]
data1 = []
# Loop through and parse the JSON to retrieve the job openings data for each year
for i in range(9):
    year_list.append(openings_response_json["Results"]["series"][0]["data"][12*i]['year'])
    data1.append(openings_response_json["Results"]["series"][0]["data"][12*i]['value'])

# Reverse the list to get the data for years in ascending order 
years = year_list[::-1]
openings = data1[::-1]

print(years)
print(openings)

In [ ]:
# Series Id for jobs hired in Health care and social assistance
series_id = "JTU620000000000000HIL"

#url
url = f"{base_url}timeseries/data/{series_id}"

#run the request using our params dictionary
hired_response = requests.get(url, params=params)

# convert response to json
hired_response_json = hired_response.json()

# Print the json
print(json.dumps(hired_response_json ,indent=4, sort_keys=True))

In [ ]:
data2 = []
# Loop through and parse the JSON to retrieve the jobs hired data for each year
for i in range(9):
    year_list.append(hired_response_json["Results"]["series"][0]["data"][12*i]['year'])
    data2.append(hired_response_json["Results"]["series"][0]["data"][12*i]['value'])
    
# Reverse the list to get the data in increasing order of years
hired = data2[::-1]

# print years and jobs hired data
print(years)
print(hired)

In [ ]:
# Series Id for for total separation
series_id = "JTU620000000000000TSL"

#url
url = f"{base_url}timeseries/data/{series_id}"

#run the request using our params dictionary
unemployed_rate_response = requests.get(url, params=params)
# convert response to json
unemployed_rate_json = unemployed_rate_response.json()

# Print the json
print(json.dumps(unemployed_rate_json ,indent=4, sort_keys=True))

In [ ]:
year_list=[]
data3 = []
# Loop through the 
for i in range(9):
    year_list.append(unemployed_rate_json["Results"]["series"][0]["data"][12*i]['year'])
    data3.append(unemployed_rate_json["Results"]["series"][0]["data"][12*i]['value'])

separations = data3[::-1]
print(years)
print(separations)

In [ ]:
data = {
    'Date': years,
    'Job Openings': openings,
    'Jobs Hired': hired,
    'Total Separations':separations
}

df = pd.DataFrame(data)
df['Job Openings'] = df['Job Openings'].astype(int)
df[ 'Jobs Hired'] = df[ 'Jobs Hired'].astype(int)
df[ 'Total Separations'] = df[ 'Total Separations'].astype(int)
# df.info()
df.set_index('Date', inplace=True)
df.plot(figsize=(12, 6))
plt.title('Healthcare Services Job Trends')
plt.xlabel('Years')
plt.ylabel('Value in thousands')
plt.grid(True)
plt.savefig("Outputs/Job_Trends_Heathcare.png")
plt.show()


In [ ]:

employment_gender = occupation_data_complete[["2022 National Employment Matrix title","Women"]] 
employment_gender = employment_gender.drop(employment_gender[employment_gender["Women"] == '–'].index)
employment_gender = employment_gender.dropna(axis=0)
employment_gender = employment_gender.fillna(0)
employment_gender["Women"] = employment_gender["Women"].astype(float)
employment_gender["Men"] = 100.0 - employment_gender["Women"] 

top_10 = employment_gender.sort_values("Women",ascending = False).head(10)
top_10.set_index("2022 National Employment Matrix title", inplace = True)
top_10.plot(kind = "bar",figsize=(10,4))


In [ ]:
gender_ratio = employment_gender.loc[employment_gender["2022 National Employment Matrix title"] == "Management Occupations"]
# Data for the pie chart
values = [gender_ratio.iloc[0,1], gender_ratio.iloc[0,2]]
labels = ["Women", "Men"]

# Create a pie chart
plt.pie(values, labels=labels, autopct='%1.1f%%', startangle=190)
plt.axis('equal')  # Equal aspect ratio ensures that the pie chart is circular.

# Display the pie chart
plt.show()


In [ ]:
# Analysis for occupation which are impacted by automation.
automation_affected_occ = pd.merge(occupation_automation,fastest_declining_occ,on=["2022 National Employment Matrix title","2022 National Employment Matrix title"])
high_automation_probabillity_occ=automation_affected_occ.loc[automation_affected_occ['Automation Probability'] > 0.90]
high_automation_probabillity_occ_sorted=high_automation_probabillity_occ.sort_values("Employment change, percent, 2022-32",ascending=True)
high_automation_probabillity_occ_sorted